In [119]:
import subprocess
SOLR_EXECUTABLE = '/usr/local/Cellar/solr/8.0.0/bin/solr'
def solr_command(*args):
    return subprocess.check_output([SOLR_EXECUTABLE] + list(args))
def create_collection(config_dir, collection_name):
    solr_command('create_core', '-c', collection_name, '-d', config_dir)

In [136]:
import os

reviews_path = f'{os.getcwd()}/amazon-reviews'
products_path = f'{os.getcwd()}/amazon-products'
create_collection(reviews_path, 'amazon-reviews')
create_collection(products_path, 'amazon-products')

In [132]:
import html
    
def product_json(filename):
    list_of_dictionaries = []
    for item in open(filename):
        dict = {}
        line = eval('(' + item + ')')
        if 'title' in line:
            dict.update({'title': html.unescape(line['title'])})
        else:
            continue
        if 'description' in line:
            dict.update({'description': html.unescape(line['description'])})
        if 'price' in line:
            dict.update({'price': float('{:.2f}'.format(line['price']))})
        dict.update({'asin': line['asin']})
        list_of_dictionaries.append(dict)
    return list_of_dictionaries

def review_json(filename, products):
    list_of_dictionaries = []
    product_asins = [x['asin'] for x in products]
    for item in open(filename):
        dict = {}
        line = eval('(' + item + ')')
        if line['asin'] in product_asins:
            dict.update({'reviewText': line['reviewText']})
            dict.update({'overall': int(line['overall'])})
            dict.update({'summary': line['summary']})
            dict.update({'asin': line['asin']})
            list_of_dictionaries.append(dict)
    return list_of_dictionaries

In [133]:
products = product_json("medium_asin_data.txt")
reviews = review_json("medium_review_data.txt", products)

In [137]:
import pysolr
def index(data, port=8983, collection_name=''):
    solr = pysolr.Solr(f'http://localhost:{port}/solr/{collection_name}')
    solr.add(data, commit=True)

In [138]:
index(reviews, collection_name='amazon-reviews')

In [145]:
index(products, collection_name='amazon-products')